## ETL Commits Data from Github REST API

From https://github.com/apache/airflow, download commits over the last 6 months.  For this POC, Mar to Aug 2022 is selected for ETL and subsequent analysis.


In [1]:
# to automatically reload jupyter whenever the code in common module is changed
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from datetime import datetime, timezone
import sys
import time
import requests
import json
import sqlite3
import pandas as pd

import common as comm
import my_credentials


sql_db = 'github_api.db'

c:\Users\aikch\anaconda3\envs\edb_assignm\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


### To download commits data

API calls are made in batches i.e., fortnightly batches from Mar to Aug 2022 to address the rate-limit imposed by Github REST API.  Pauses are made between calls to throttle the call frequencies as well.

It takes between 14 to 15mins to collect all responses for the six month periods, using home wifi.

In [2]:
six_mth_period = [('2022-03-01', '2022-03-14'),
                ('2022-03-15', '2022-03-27'),
                ('2022-03-28', '2022-04-09'),
                ('2022-04-10', '2022-04-22'),
                ('2022-04-23', '2022-05-05'),
                ('2022-05-06', '2022-05-18'),
                ('2022-05-19', '2022-05-31'),
                ('2022-06-01', '2022-06-13'),
                ('2022-06-14', '2022-06-26'),
                ('2022-06-27', '2022-07-09'),
                ('2022-07-10', '2022-07-22'),
                ('2022-07-23', '2022-08-05'),
                ('2022-08-06', '2022-08-18'),
                ('2022-08-19', '2022-08-31'),]
                

access_token = my_credentials.secret_token.get('access_token', None)
headers = {'Authorization': access_token}
base_query_url = 'https://api.github.com/search/commits?q=repo:apache/airflow'

response_collected = comm.collect_data(base_query_url, headers, six_mth_period)


### To write data to sqlite3 database

In [3]:
# create table
conn = sqlite3.connect('github_api.db')
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS commits_history
    ([committer_name] TEXT,  
     [committer_email] TEXT,
     [commit_datetime] TEXT,
     [commit_url] TEXT)
""")  
conn.commit()
conn.close()


In [4]:
# delete data b4 insert of data
conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
query = """
    DELETE FROM commits_history
"""
cursor.execute(query)
conn.commit()


In [5]:
# insert data
conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
query = """
    INSERT INTO commits_history
    (committer_name, committer_email, commit_datetime, commit_url)
    VALUES
    (?, ?, ?, ?);    
"""

for period, list_of_dicts in response_collected.items():
    for dict_obj in list_of_dicts['items']:
        try:
            datapoints = (
            dict_obj['commit']['committer']['name'],
            dict_obj['commit']['committer']['email'], 
            dict_obj['commit']['committer']['date'],
            dict_obj['commit']['url']
            ) 
        except TypeError as e:  # TypeError: 'NoneType' object is not subscriptable
            pass
    cursor.execute(query, datapoints)  # ingest only complete records, ignoring NoneType
    conn.commit()


In [6]:
# read data
conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
cursor.execute("""
    SELECT * 
    FROM commits_history
""")



df = pd.DataFrame(cursor.fetchall(), columns=['committer_name', 'committer_email', 'commit_datetime', 'commit_url'])
conn.close()


print(df.head(30))
print(f"df.shape: {df.shape}")

   committer_name     committer_email                commit_datetime  \
0          GitHub  noreply@github.com  2022-03-01T15:28:28.000+01:00   
1          GitHub  noreply@github.com  2022-03-21T21:38:25.000+01:00   
2          GitHub  noreply@github.com  2022-04-08T18:20:54.000+01:00   
3          GitHub  noreply@github.com  2022-04-12T20:15:29.000+01:00   
4    Jarek Potiuk    jarek@potiuk.com  2022-04-25T23:05:00.000+02:00   
5          GitHub  noreply@github.com  2022-05-16T14:13:53.000-07:00   
6          GitHub  noreply@github.com  2022-05-24T08:10:18.000-06:00   
7          GitHub  noreply@github.com  2022-06-07T13:13:16.000+02:00   
8          GitHub  noreply@github.com  2022-06-24T16:51:01.000+01:00   
9          GitHub  noreply@github.com  2022-07-05T12:43:43.000+02:00   
10         GitHub  noreply@github.com  2022-07-11T09:52:23.000-06:00   
11         GitHub  noreply@github.com  2022-08-04T11:05:52.000-04:00   
12         GitHub  noreply@github.com  2022-08-18T23:13:21.000+0